# Finance Benchmark

This Notebook was design for explaining the benchmark part of the finances packages. The benchamark is composed of the following code:

1. **probability_class.py**: Code for dealing with diffierent probability densities. 
2. **payoff_class.py**: Code for dealing with different pay  offs
3. **finance_benchamark.py**:  Code for solving a specific finance problem using a specific *amplitude estimation* algorithm

In [ ]:
import sys
sys.path.append("../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
import ast
def get_circuit_staff(input_string):
    
    circuit_stats = pd.DataFrame(ast.literal_eval(input_string)).T
    
    circuit_stats = pd.concat(
        [
            circuit_stats, 
            pd.DataFrame(list(circuit_stats["gates"].values), index=circuit_stats["gates"].index)
        ],
        axis=1
    )
    circuit_stats.drop(['gates'], axis=1, inplace=True)
    circuit_stats.rename(columns = {'nbqbits': 'total_n_qbits'}, inplace=True)
    return circuit_stats

## 1. probability_class.py

In this script the **DensityProbability** class is defined. The only mandatory input for this class is:

* *probability_type*: string with the type of probability density to load. (*Black-Scholes*)

The different parameters for the probability density should be provided as a dictionary. The parameters should be defined according to the definition of the probability density function desired.

Additionally the main atribute of the class will be the **probability**. This property is the desired probability density where the parameters provided to the class are fixed.

In [ ]:
from probability_class import DensityProbability

In [ ]:
#Configuration  of a probability density
probability_type = "Black-Scholes"

density_dict = {
    "s_0": 2.0,
    "risk_free_rate": 0.05,
    "maturity": 0.5,
    "volatility": 0.5    
}


In [ ]:
bs_pdf = DensityProbability(probability_type, **density_dict)

As can be seen the the *probability* property of the class is a function (in fact is a python partial function of the probability density desired).

In [ ]:
type(bs_pdf.probability)

In the case of the example we have configured a **Black-Scholes** probability density with the parameters povided in the *density_dict*. Now we can plot the distribution over a domain!

In [ ]:
x = np.linspace(0.1, 6.0, 2**9)
plt.plot(x, bs_pdf.probability(x))

In [ ]:
#Playing with Black-Scholes

list_of_functions = []
#Lista = [1.0, 2.0, 3.0] #for s_0
#Lista = [0.2, 0.4, 0.6, 0.8, 1.0, 1.2] #for maturity
#Lista = [0.01, 0.02, 0.03, 0.04, 0.05] #for risk_free_rate
Lista = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5] #for volatility

for i in Lista:
    step_dict = density_dict
    #step_dict.update({"s_0": i})
    #step_dict.update({"maturity": i})
    step_dict.update({"volatility": i})
    #step_dict.update({"risk_free_rate": i})
    step_c = DensityProbability(probability_type, **step_dict) 
    list_of_functions.append(step_c.probability)
x = np.linspace(0.1, 3.0, 100)
for func in list_of_functions:
    plt.plot(x, func(x))
plt.legend(Lista)    

In [ ]:
rfr_factor = np.exp(-density_dict['risk_free_rate']*density_dict['maturity'])
print("Factor form risk free rate: ", rfr_factor)

## 2. payoff_class.py

In this script the  **PayOff** class is defined. For this class the input is a dictionary where different keys can be provided. Most important one is:

* *pay_off_type*: string with the type of payoff desired. The options are:
    * European_Call_Option
    * European_Put_Option
    * Digital_Call_Option
    * Digital_Put_Option
    * Futures

The other keys of the dictionary are used for configuring the desired payoff. The payoffs are obtained from **QQuantLib/utils/classical_finance** module. So the keys of the input dictionary should be the same keys needed for configuring the payoffs in the before module.

The class created the following 2 properties:

* **pay_off**: function with the desired payoff and the propper pay off configuration given by the input dictionary
* **pay_off_bs**: gives the exact price of the payoff under the **Black-Scholes** model


In [ ]:
from payoff_class import PayOff

### 2.1 European_Call_Option

In [ ]:
eco_payoff = {
    "pay_off_type": "European_Call_Option",
    "strike": 0.5,
}

eco = PayOff(**eco_payoff)

As can be seen the two properties are python partial functions where the payoff parameters are fixed

In [ ]:
print(type(eco.pay_off))
print(type(eco.pay_off_bs))

In [ ]:
x = np.linspace(0.1, 7.0, 2**9)
plt.plot(x, eco.pay_off(x))

In [ ]:
plt.plot(x, bs_pdf.probability(x))

For using the *pay_off_bs* function we need to provided the **Black-Scholes** configuration.

In [ ]:
print("Pay Off price unde BS model: ", eco.pay_off_bs(**density_dict))
print("Classical pay Off: ", np.sum(eco.pay_off(x, **density_dict)*bs_pdf.probability(x))*rfr_factor)

### 2.2 European_Put_Option

In [ ]:
epo_payoff = {
    "pay_off_type": "European_Put_Option",
    "strike": 1.5,
}

epo = PayOff(**epo_payoff)

As can be seen the two properties are python partial functions where the payoff parameters are fixed

In [ ]:
x = np.linspace(0.1, 5.5, 2**7)
plt.plot(x, epo.pay_off(x))

In [ ]:
plt.plot(x, bs_pdf.probability(x))

In [ ]:
print("Pay Off price unde BS model: ", epo.pay_off_bs(**density_dict))
print("Classical pay Off: ", np.sum(epo.pay_off(x, **density_dict)*bs_pdf.probability(x))*rfr_factor)

### 2.3 Digital_Call_Option

In [ ]:
dco_payoff = {
    "pay_off_type": "Digital_Call_Option",
    "strike": 0.5,
    "coupon": 1.0
}

dco = PayOff(**dco_payoff)

As can be seen the two properties are python partial functions where the payoff parameters are fixed

In [ ]:
x = np.linspace(0.1, 4.0, 2**7)
plt.plot(x, dco.pay_off(x))

In [ ]:
plt.plot(x, bs_pdf.probability(x))

In [ ]:
print("Pay Off price unde BS model: ", dco.pay_off_bs(**density_dict))
print("Classical pay Off: ", np.sum(dco.pay_off(x, **density_dict)*bs_pdf.probability(x))*rfr_factor)

### 2.4 Digital_Put_Option

In [ ]:
dpo_payoff = {
    "pay_off_type": "Digital_Put_Option",
    "strike": 1.5,
    "coupon": 1.0    
}

dpo = PayOff(**dpo_payoff)

As can be seen the two properties are python partial functions where the payoff parameters are fixed

In [ ]:
x = np.linspace(0.1, 5.5, 2**9)
plt.plot(x, dpo.pay_off(x))

In [ ]:
plt.plot(x, bs_pdf.probability(x))

In [ ]:
print("Pay Off price unde BS model: ", dpo.pay_off_bs(**density_dict))
print("Classical pay Off: ", np.sum(dpo.pay_off(x)*bs_pdf.probability(x))*rfr_factor)

### 2.5 Futures

In [ ]:
future_po_dict = {
    "pay_off_type": "Futures",
    "strike": 1.5,   
}

future = PayOff(**future_po_dict)

In [ ]:
x = np.linspace(0.1, 5.5, 2**9)
plt.plot(x, future.pay_off(x))

In [ ]:
plt.plot(x, bs_pdf.probability(x))

In [ ]:
print("Pay Off price unde BS model: ", future.pay_off_bs(**density_dict))
print("Classical pay Off: ", np.sum(future.pay_off(x)*bs_pdf.probability(x))*rfr_factor)

## 3. finance_benchmark.py

The finance_benchmark.py script creates the **PriceEstimation** class. This class will solve what we call an **Amplitude Estimation Price Problem** (**AE_PriceP** from now). This is: it will compute the price of a input payoff, under a probability density for a domain input using a properly configured amplitude estimation method (using one of the different **amplitude estimation** algorithm available from package **QQuantLib/AE**).

The input of the class will be a python dictionary (that we will call **AE_PriceP** dictionary or problem from now) where the configuration of the price problem (payoff, probability density and domain) and the desired **amplitude estimation** algorithm should be provided. The input dictionary will be a big dictionary with a lot of keys. The key for selecting the *amplitude estimation* algorithm is:

* **ae_type**: posible values will be:
    * *MLAE*
    * *CQPEAE*
    * *IQPEAE*
    * *IQAE*
    * *RQAE*

When the class is instantiated following steps are followed:

1. Create a domain (x) using the corresponding keys of the input dictionary. The domain will be the x, so we are going to define an interval between $[x_0, x_f]$ divide en $N=2^{n_{qbits}}$ parts. For configurate a domain following keys are used:
    * x0: initial value of the domain 
    * xf: final value of the domain
    * n_qbits: for setting the number of parts the domain interval will be splited: $2^{n\_qbits}$
2. Created a density probability class using: **DensityPobability** from *probability_class* module and the corresponding keys of the input dicitionary. Using the class and the domain from step 1 the probability numpy array is created (property **probability**)
3. Create a payoff class using: **PayOff** from *payoff_class* module and the corresponding keys of the input dicitionary. Using the class and the domain from step 1 the payoff numpy array is created (property **pay_off**)
4. If necesary the class populates the *pay_off_normalised* atribute used for doing payoff normalisation.

The class have a **run** method for solving the input **AE_PriceP** problem. The main steps of this method are:

1. Execution of the *create_oracle* method. This method creates the mandatory quantum oracle (property: **derivative_oracle**) needed by the **amplitude estimation** algorithm. The *create_oracle* method can implement two types of probability density loading depending on the input dictionary key: **probability_loading**:
    1. probability_loading: True. The probability density will be loading as a pure probability density (the uinform distribution will be replaced by the input probability density).
    2. probability_loading: False. The probability density will be loading as a function over an extra qbit.
2. method: *run_ae*: execute a *amplitude estimation* algorithm using the **derivative_oracle**. The executed algorithm will be defined by the key *ae_type* of the input dictionaty. For the configuration of the *amplitude estiamtion* methods will be used the different keys of the input dictionary. For each method the keys sholud be consulted in the propper algorithm module of the **QQuantLib/AE/** package

The **run** method execute the desired *amplitude estimation* algorithm over the created price estimation problem a defined number of times (key: *number_of_tests* from the input dictionary).

Finally the **run** methdos stores the important information from the solution in the **pdf** property

### 3.1 Example-01

In [ ]:
#Example of AE_PriceP dictionary
m_k = [1, 100, 150, 200, 250, 300, 310, 320]
ae_pricep = {
    #Amplitude Estimation selection
    'ae_type': 'MLAE',
    
    #Amplitude Estimation configuration
    'schedule': [
        m_k,
        [100 for i in m_k]
    ],
    'mcz_qlm': False,
    'delta' : 1.0e-6,
    'ns' : 10000,
    'auxiliar_qbits_number': None,
    'cbits_number': None,
    'alpha': None,
    'gamma': None,
    'epsilon': None,
    'shots': None,
    
    #Loading Probability
    'probability_loading': False,
    
    #Numbe of problem to solve
    'number_of_tests': 1,
    
    #PayOff Configuration
    'pay_off_type': 'European_Call_Option',
    'strike': 0.5,
    'coupon': None,
    #Domain configuration
    'x0': 0.01,
    'xf': 3.5,
    'n_qbits': 5,
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.5,
    'save': False
}

In [ ]:
from finance_benchmark import PriceEstimation

In [ ]:
price_estimation = PriceEstimation(**ae_pricep)

In [ ]:
%%time
price_estimation.run()

In [ ]:
price_estimation.pdf

In [ ]:
Columnas = ["ae_type", "pay_off_type", "probability_type", "s_0", "maturity", "volatility", "n_qbits",
    "classical_price_rfr", "derivative_price_rfr_ae", "exact_solution"]
price_estimation.pdf[Columnas]

In [ ]:
Columnas = ["ae_type", "pay_off_type", "probability_type", "s_0", "maturity", "volatility", "n_qbits",
    "relative_error_classical", "relative_error_exact"]
price_estimation.pdf[Columnas]

In [ ]:
stats_for_circuit = pd.concat(list(price_estimation.pdf["circuit_stasts"].apply(
    lambda x: get_circuit_staff(str(x))
)))


In [ ]:
stats_for_circuit

## 4. The epsilon problem.

For **IQAE** and **RQAE** an input **epsilon** for create upper and lower limits of the **amplitude estimation** is mandatory.

Main problem is that the **epsilon** only affects to the amplitude estimation problem, not to the final price value. It can happen that we provide a low **epsilon** for the amplitude estimation problem but the final price obtained has a bigger error than the desired one.

Foloowing subsections explain the problem


### 4.1 Error propagation in RQAE 

For the **RQAE** algorithm we have: 

$$\sum_{i=0}^{2^{n}-1}p(i)f(i) = 2^n * a$$

where $a$ is the value estimated by the algorithm and $\sum_{i=0}^{2^{n}-1}p(i)f(i) = z$ is the desired quantity we want to compute. The RQAE give us two limits for the $a$ value so:

$$a \in [a_l, a_u]$$

If we define the error in the estimation as:

$$\epsilon = \frac{a_u-a_l}{2} \$$

Then we can propagate the error in $a$ to the error in z:

$$\delta z = 2^n *\epsilon$$

Additionally we know that the pay_off must to be normalised so for getting the True result from z:

$$PriceEstimation = z*payoff_{normalisation}$$

So we need to propagate the $\delta z$:

$$\delta_{PriceEstimation} = \delta z * payoff_{normalisation}$$

And finally if we want to take into account the **risk free rate**:

$$Price_{derivative} = PriceEstimation * e^{-r*maturity}$$

So propagating again:

$$\delta Price_{derivative} = \delta_{PriceEstimation} * e^{-r*maturity} $$

We can create following final formula:


$$\delta Price_{derivative} = payoff_{normalisation} * e^{-r*maturity} \delta z= payoff_{normalisation} * e^{-r*maturity} *2^n *\epsilon$$


For example:
* n=5
* $e^{-r*maturity}=0.95$
* $payoff_{normalisation}=3$ 
* $\epsilon=0.01$ 

The error on the derivative price due to the esitmation intervals will be:

$\delta Price_{derivative} = 0.9$

So in this case an a priori low $\epsilon$ give us a high indetermination in the price estimation.

if we want a fixed error in the price derivative we need to invert the last equation so the $\epsilon$ we need to provide to the **RQAE** will be:

$$\epsilon = \frac{\delta Price_{derivative}}{payoff_{normalisation} * e^{-r*maturity} *2^n} \quad (1)$$ 


### 4.2 Error propagation in IQAE 

#### Density loading as an array

When the probability density is loaded as an array instead as probability for the amplitude estimation algorithms we have: 


$$\sum_{i=0}^{2^{n}-1}p(i)f(i) = 2^n * \sqrt{a}$$

where $a$ is the value estimated by the algorithm and $\sum_{i=0}^{2^{n}-1}p(i)f(i) = z$ is the desired quantity we want to compute. The IQAE give us two limits for the $a$ value so:

$$a \in [a_l, a_u]$$

If we define the error in the estimation as:

$$\epsilon = \frac{a_u-a_l}{2} \$$

We can propagate the $\epsilon$ for get an error for $z$:

$$\delta z = \frac{2^n}{2\sqrt{a}} *\epsilon$$

The payoff normalisation and the risk free rate part is the same that in the 4.1 subsection so for **IQAE** when probability density is loaded as an array  the error in the final price will be:

$$\delta Price_{derivative} = payoff_{normalisation} * e^{-r*maturity} \delta z = payoff_{normalisation} * e^{-r*maturity} \frac{2^n}{2\sqrt{a}} *\epsilon$$

So if we want a fixed error in the price derivative we need to invert the last equation so the $\epsilon$ we need to provide to the **IQAE** will be:

$$\epsilon = \frac{\delta Price_{derivative}}{payoff_{normalisation} * e^{-r*maturity}} * \frac{2\sqrt{a}}{2^n} \quad (2)$$


#### Density loading as probability density

When the probability density is loaded as a pure density for the amplitude estimation algorithms we have:

$$ a = \sum_{i=0}^{2^{n}-1}\left|\sqrt{p(x_i)f(x_i)}\right|^2 = \sum_{i=0}^{2^{n}-1}\left|p(x_i)f(x_i)\right|$$

where $a$ is the value estimated by the algorithm and $\sum_{i=0}^{2^{n}-1}\left|p(x_i)f(x_i)\right| = z$ is the desired quantity we want to compute. The IQAE give us two limits for the $a$ value so:

$$a \in [a_l, a_u]$$

If we define:

$$\epsilon = \frac{a_u-a_l}{2} \$$

We can propagate the $\epsilon$ for get an error for $z$:

$$\delta z = \epsilon$$

The payoff normalisation and the risk free rate part is the same that in the 4.1 subsection so for **IQAE** when probability density is loaded as a purde density the error in the final price will be:

$$\delta Price_{derivative} = payoff_{normalisation} * e^{-r*maturity} * \delta z = payoff_{normalisation} * e^{-r*maturity} * \epsilon$$

So if we want a fixed error in the price derivative we need to invert the last equation so the $\epsilon$ we need to provide to the **IQAE** will be:

$$\epsilon = \frac{\delta Price_{derivative}}{payoff_{normalisation} * e^{-r*maturity}} \quad (3)$$

The **PriceEStimation** class use the **epsilon** for amplitude estimation. Additionally computes the propagated error for price derivative that will be accesible using **delta_price**.


The **PriceEStimation** class do this error propagation. So the input **epsilon** will be in fact the $\delta Price_{derivative}$. In fact in the pdf attribute of the class there will be the following columns:


In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
from finance_benchmark import PriceEstimation

## RQAE

In [ ]:
#Example of complete dictionary
problem_dict = {
    #Amplitude Estimation selection
    'ae_type': 'RQAE',
    
    #Amplitude Estimation configuration
    'mcz_qlm': False,    
    'schedule': None,
    'delta' : None,
    'auxiliar_qbits_number': None,
    'cbits_number': None,
    'alpha': None,
    'gamma': 0.05,
    'epsilon': 0.001,
    'shots': 100,
    
    #Loading Probability
    'probability_loading': False,
    
    #Numbe of problem to solve
    'number_of_tests': 1,
    
    #PayOff Configuration
    'pay_off_type': 'European_Call_Option',
    'strike': 0.5,
    'coupon': None,
    
    #Domain configuration
    'x0': 0.01,
    'xf': 3.5,
    'n_qbits': 5,
    
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.5,
    
    'save': False
}

In [ ]:
price_estimation = PriceEstimation(**problem_dict)

In [ ]:
price_estimation.run()

In [ ]:
#Expected maximum error for the price derivative
print("delta Price derivative: ", str(price_estimation.pdf['delta_price'].iloc[0]))
#Pure epsilon for the amplitude estimation
print("Epsilon for AE algorithm : ", str(price_estimation.pdf['epsilon'].iloc[0]))

In [ ]:
#Comparison should be respect classical_price not to exact solution
np.abs(
    price_estimation.pdf['classical_price_rfr'] - price_estimation.pdf['derivative_price_rfr_ae']
)/2.0 < price_estimation.pdf['delta_price'] 

In [ ]:
price_estimation.pdf['classical_price_rfr'] - price_estimation.pdf['derivative_price_rfr_ae']

In [ ]:
#Original epsilon for amplitude estimation
(price_estimation.pdf['ae_u']-price_estimation.pdf['ae_l'])/2.0

In [ ]:
price_estimation.pdf['epsilon']

In [ ]:
(price_estimation.pdf['derivative_price_rfr_ae_u']-price_estimation.pdf['derivative_price_rfr_ae_l'])/2.0

In [ ]:
price_estimation.pdf['delta_price']

## IQAE

In [ ]:
#Example of complete dictionary
problem_dict = {
    #Amplitude Estimation selection
    'ae_type': 'IQAE',
    
    #Amplitude Estimation configuration
    'mcz_qlm': False,    
    'schedule': None,
    'delta' : None,
    'auxiliar_qbits_number': None,
    'cbits_number': None,
    'alpha': 0.05,
    'gamma': None,
    'epsilon': 0.05,
    'shots': 100,
    
    #Loading Probability
    'probability_loading': True,
    
    #Numbe of problem to solve
    'number_of_tests': 1,
    
    #PayOff Configuration
    'pay_off_type': 'European_Call_Option',
    'strike': 0.5,
    'coupon': None,
    
    #Domain configuration
    'x0': 0.01,
    'xf': 5.0,
    'n_qbits': 5,
    
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.5,
    
    'save': False    
}

In [ ]:
price_estimation = PriceEstimation(**problem_dict)

In [ ]:
price_estimation.run()

In [ ]:
#Expected maximum error for the price derivative
print("delta Price derivative: ", str(price_estimation.pdf['delta_price'].iloc[0]))
#Pure epsilon for the amplitude estimation
print("Epsilon for AE algorithm : ", str(price_estimation.pdf['epsilon'].iloc[0]))

In [ ]:
#Original epsilon for amplitude estimation
(price_estimation.pdf['ae_u']-price_estimation.pdf['ae_l'])/2.0

In [ ]:
price_estimation.pdf['epsilon']

In [ ]:
(price_estimation.pdf['derivative_price_rfr_ae_u']-price_estimation.pdf['derivative_price_rfr_ae_l'])/2.0

In [ ]:
price_estimation.pdf['delta_price']

In [ ]:
stats_for_circuit = pd.concat(list(price_estimation.pdf["circuit_stasts"].apply(
    lambda x: get_circuit_staff(str(x))
)))
stats_for_circuit

### Loading probability as an array

In [ ]:
#Example of complete dictionary
problem_dict = {
    #Amplitude Estimation selection
    'ae_type': 'IQAE',
    
    #Amplitude Estimation configuration
    'mcz_qlm': False,    
    'schedule': None,
    'delta' : None,
    'auxiliar_qbits_number': None,
    'cbits_number': None,
    'alpha': 0.05,
    'gamma': None,
    'epsilon': 0.0005,
    'shots': 100,
    
    #Loading Probability
    'probability_loading': False,
    
    #Numbe of problem to solve
    'number_of_tests': 1,
    
    #PayOff Configuration
    'pay_off_type': 'European_Call_Option',
    'strike': 0.5,
    'coupon': None,
    
    #Domain configuration
    'x0': 0.01,
    'xf': 5.0,
    'n_qbits': 5,
    
    #Probability density configuration
    'probability_type': 'Black-Scholes',
    's_0': 1,
    'risk_free_rate': 0.05,
    'maturity': 1.0,
    'volatility': 0.5,

    'save': False 
}

#problem_dict.update({"qpu": linalg_qpu})

In [ ]:
price_estimation = PriceEstimation(**problem_dict)

In [ ]:
price_estimation.run()

In [ ]:
#Expected maximum error for the price derivative
print("delta Price derivative: ", str(price_estimation.pdf['delta_price'].iloc[0]))
#Pure epsilon for the amplitude estimation
print("Epsilon for AE algorithm : ", str(price_estimation.pdf['epsilon'].iloc[0]))

In [ ]:
#Original epsilon for amplitude estimation
(price_estimation.pdf['ae_u']-price_estimation.pdf['ae_l'])/2.0

In [ ]:
price_estimation.pdf['epsilon']

In [ ]:
(price_estimation.pdf['derivative_price_rfr_ae_u']-price_estimation.pdf['derivative_price_rfr_ae_l'])/2.0

In [ ]:
price_estimation.pdf['delta_price']

In [ ]:
price_estimation.pdf[['derivative_price_rfr_ae', 'classical_price_rfr']]

In [ ]:
stats_for_circuit = pd.concat(list(price_estimation.pdf["circuit_stasts"].apply(
    lambda x: get_circuit_staff(str(x))
)))
stats_for_circuit

In [ ]:
price_estimation.pdf['relative_error_classical']